In [1]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_community.document_loaders import GitLoader
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain_community.document_loaders import WebBaseLoader
from git import Repo
# LCEL docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
repo = Repo.clone_from(
    "https://github.com/langchain-ai/langgraph", to_path="../../../../repos/langgraph/"
)
branch = repo.head.reference

#loader_langchain_code = GitLoader(repo_path="../../../../repos/langchain/", branch="master")

loader_langgraph_code = GitLoader(repo_path="../../../../repos/langgraph/", branch=branch)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/langchain-ai/langgraph ../../../../repos/langgraph/
  stderr: 'fatal: destination path '../../../../repos/langgraph' already exists and is not an empty directory.
'

In [7]:
url = "https://langchain-ai.github.io/langgraph/"
recursive_url_loader = RecursiveUrlLoader(
    url=url, max_depth=500, extractor=lambda x: Soup(x, "html.parser").text
)

loader_multiple_pages = WebBaseLoader(["https://langchain-ai.github.io/langgraph/tutorials/introduction/", 
                                       "https://langchain-ai.github.io/langgraph/tutorials/code_assistant/langgraph_code_assistant/", 
                                       "https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag_local/",
                                       "https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams/", 
                                       "https://langchain-ai.github.io/langgraph/tutorials/storm/storm/", 
                                       "https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/", 
                                       "https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/", ""])


loader_all = MergedDataLoader(loaders=[recursive_url_loader])

docs = loader_all.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

Unable to load from https://langchain-ai.github.io/langgraph/how-tos/subgraph/. Received error HTTPSConnectionPool(host='langchain-ai.github.io', port=443): Read timed out. (read timeout=10) of type ReadTimeout


MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.02,  # <-- Super slow! We can only make a request once every 10 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
)

C:\Users\hmazomba\AppData\Local\Temp\ipykernel_22284\1506048698.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

google_api__key = os.getenv('GOOGLE_API_KEY')
### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in LangChain, a framework for developing applications powered by language models. \n 
    Here is a full set of LangChain documentation:  \n ------- \n  {context} \n ------- \n Answer the user 
    question based on the above provided documentation. Ensure any code you provide can be executed \n 
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# Data model
class code(BaseModel):
    """Schema for code solutions to questions about LCEL."""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-exp-0827", rate_limiter=rate_limiter)
code_gen_chain_oai = code_gen_prompt | llm.with_structured_output(code)
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain_oai.invoke({"context":concatenated_content,"messages":[("user",question)]})

c:\Users\hmazomba\miniforge3\envs\kate\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(solution)

prefix="Here is a basic RAG chain built in LCEL using Anthropic\\'s Claude model and FAISS as a vector store:\\n\\n1.  Load the documents and split into chunks\\n2.  Create the vector store\\n3.  Create the retriever\\n4.  Create the prompt template\\n5.  Create the chain" imports='from langchain_core.prompts import ChatPromptTemplate\\nfrom langchain_core.output_parsers import StrOutputParser\\nfrom langchain_community.vectorstores import FAISS\\nfrom langchain_community.embeddings import HuggingFaceEmbeddings\\nfrom langchain_core.retrievers import VectorStoreRetriever\\nfrom langchain_anthropic import ChatAnthropic' code='from langchain_core.prompts import ChatPromptTemplate\\nfrom langchain_core.output_parsers import StrOutputParser\\nfrom langchain_community.vectorstores import FAISS\\nfrom langchain_community.embeddings import HuggingFaceEmbeddings\\nfrom langchain_core.retrievers import VectorStoreRetriever\\nfrom langchain_anthropic import ChatAnthropic\\n\\n# Load the document

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

### Anthropic

# Prompt to enforce tool use
code_gen_prompt_gemini = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user  question based on the \n 
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined. Structure your answer: 1) a prefix describing the code solution, 2) the imports, 3) the functioning code block. \n
    Invoke the code tool to structure the output correctly.  \n Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)

structured_llm_gemini = llm.with_structured_output(code, include_raw=True)


# Optional: Check for errors in case tool use is flaky
def check_claude_output(tool_output):
    """Check for parse error or failure to call the tool"""

    # Error with parsing
    if tool_output["parsing_error"]:
        # Report back output and parsing errors
        print("Parsing error!")
        raw_output = str(tool_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"Error parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        )

    # Tool was not invoked
    elif not tool_output["parsed"]:
        print("Failed to invoke tool!")
        raise ValueError(
            "You did not use the provided tool! Be sure to invoke the tool to structure the output."
        )
    return tool_output


# Chain with output check
code_chain_gemini_raw = (
    code_gen_prompt_gemini | structured_llm_gemini | check_claude_output
)


def insert_errors(inputs):
    """Insert errors for tool parsing in the messages"""

    # Get errors
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "assistant",
            f"Retry. You are required to fix the parsing errors: {error} \n\n You must invoke the provided tool.",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }


# This will be run as a fallback chain
fallback_chain = insert_errors | code_chain_gemini_raw
N = 3  # Max re-tries
code_gen_chain_re_try = code_chain_gemini_raw.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)


def parse_output(solution):
    """When we add 'include_raw=True' to structured output,
    it will return a dict w 'raw', 'parsed', 'parsing_error'."""

    return solution["parsed"]


# Optional: With re-try to correct for failure to invoke tool
code_gen_chain = code_gen_chain_re_try | parse_output

# No re-try
code_gen_chain = code_gen_prompt_gemini | structured_llm_gemini | parse_output

In [ ]:
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)   
print(solution)

prefix='Here is how you can build a RAG chain in LCEL:\\n\\n1. **Initialize Necessary Components:**\\n   - Choose an LLM (e.g., ChatOpenAI).\\n   - Set up a vector store with documents and embeddings (e.g., FAISS with OpenAI embeddings).\\n   - Create a prompt template to guide the LLM in using the retrieved context.\\n\\n2. **Create the Chain:**\\n   - Use the `|` operator to chain the components together:\\n     - `retriever | prompt | llm | StrOutputParser`\\n\\n**Example:**' imports='from langchain_core.prompts import ChatPromptTemplate\\nfrom langchain_core.output_parsers import StrOutputParser\\nfrom langchain_community.vectorstores import FAISS\\nfrom langchain_community.embeddings import OpenAIEmbeddings\\nfrom langchain_openai import ChatOpenAI' code='from langchain_core.prompts import ChatPromptTemplate\\nfrom langchain_core.output_parsers import StrOutputParser\\nfrom langchain_community.vectorstores import FAISS\\nfrom langchain_community.embeddings import OpenAIEmbeddings\

In [ ]:
from typing import List, TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error: str
    messages: List
    generation: str
    iterations: int

In [ ]:
### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = "do not reflect"

### Nodes


def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection

    # Add reflection
    reflections = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


### Edges


def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")
app = workflow.compile()

In [ ]:
question = "How can I make Hierachical team made of a SEO Expert, UI/UX Expert, Content Developer and Copywriter using the langgraph framework?"
solution = app.invoke({"messages": [("user", question)], "iterations": 0, "error":""})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---


In [ ]:
solution['generation']

code(prefix='RunnablePassthrough takes a string as input and passes it directly to the next runnable in the chain. You can use this to construct a prompt that uses the input string directly.', imports='from langchain_core.prompts import PromptTemplate\\nfrom langchain_core.runnables import RunnablePassthrough', code='prompt = PromptTemplate.from_template("The input is: {input}")\\nchain = RunnablePassthrough() | prompt\\nprint(chain.invoke("my input string"))')